# Imports and Setups

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import models

import matplotlib.pyplot as plt 
import numpy as np
import random
import time
import os

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

### W&B - Experiment Tracking

In [2]:
%%capture
!pip install wandb 

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

## Dataset gathering and preparation

Only **10%** labeled data is being used. 

In [4]:
# Gather Flowers dataset
train_ds, extra_train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


## ResNet50 base and a custom classification head

In [5]:
def get_training_model(trainable=False):
    inputs = layers.Input(shape=(224, 224, 3))
    EXTRACTOR = tf.keras.applications.ResNet50(weights="imagenet", include_top=False,
        input_shape=(224, 224, 3))
    EXTRACTOR.trainable = trainable
    x = EXTRACTOR(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(5, activation="softmax")(x)
    classifier = models.Model(inputs=inputs, outputs=x)
    
    return classifier

In [6]:
model = get_training_model()
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


## Callback

In [7]:
# Early Stopping to prevent overfitting
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

## Without Augmentation

### Warm Up

In [9]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(training_ds,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
6/6 [==============================] - 3s 513ms/step - loss: 1.8391 - acc: 0.2044 - val_loss: 1.6250 - val_acc: 0.2055
Epoch 2/35
6/6 [==============================] - 2s 320ms/step - loss: 1.6024 - acc: 0.2888 - val_loss: 1.7046 - val_acc: 0.2145
Epoch 3/35
6/6 [==============================] - 2s 359ms/step - loss: 1.6264 - acc: 0.3079 - val_loss: 1.6490 - val_acc: 0.2436
Epoch 00003: early stopping


In [10]:
model.save('warmup.h5')

### Fine tune CNN

In [11]:
# prepare model and compile
model.layers[1].trainable = True

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer=tf.keras.optimizers.Adam(1e-5))

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(training_ds,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
6/6 [==============================] - 6s 924ms/step - loss: 1.5775 - acc: 0.3025 - val_loss: 1.5856 - val_acc: 0.2527
Epoch 2/35
6/6 [==============================] - 3s 507ms/step - loss: 1.5114 - acc: 0.3542 - val_loss: 1.6024 - val_acc: 0.2491
Epoch 3/35
6/6 [==============================] - 3s 519ms/step - loss: 1.4678 - acc: 0.3651 - val_loss: 1.6279 - val_acc: 0.2527
Epoch 00003: early stopping


### Evaluation

In [13]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 1s 107ms/step - loss: 1.5856 - acc: 0.2527


# Training with Augmentation


### Augmentation

In [14]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [19]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [16]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

### Warmup

In [17]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(trainloader,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
6/6 [==============================] - 3s 509ms/step - loss: 1.8788 - acc: 0.2916 - val_loss: 1.6496 - val_acc: 0.2055
Epoch 2/35
6/6 [==============================] - 2s 400ms/step - loss: 1.6126 - acc: 0.2343 - val_loss: 1.6411 - val_acc: 0.2291
Epoch 3/35
6/6 [==============================] - 3s 452ms/step - loss: 1.6228 - acc: 0.2561 - val_loss: 1.6226 - val_acc: 0.2145
Epoch 4/35
6/6 [==============================] - 3s 476ms/step - loss: 1.5738 - acc: 0.2970 - val_loss: 1.6112 - val_acc: 0.2200
Epoch 5/35
6/6 [==============================] - 3s 450ms/step - loss: 1.5685 - acc: 0.3106 - val_loss: 1.5797 - val_acc: 0.2764
Epoch 6/35
6/6 [==============================] - 3s 463ms/step - loss: 1.5490 - acc: 0.3433 - val_loss: 1.5661 - val_acc: 0.2745
Epoch 7/35
6/6 [==============================] - 3s 459ms/step - loss: 1.5447 - acc: 0.3134 - val_loss: 1.5596 - val_acc: 0.2582
Epoch 8/35
6/6 [==============================] - 2s 410ms/step - loss: 1.5371 - acc: 0.33

In [18]:
model.save('warmup_augmentation.h5')

### Fine tune CNN

In [20]:
# prepare model and compile
model.layers[1].trainable = True

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer=tf.keras.optimizers.Adam(1e-5))

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(trainloader,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
6/6 [==============================] - 5s 835ms/step - loss: 1.5755 - acc: 0.3488 - val_loss: 1.5523 - val_acc: 0.2655
Epoch 2/35
6/6 [==============================] - 4s 716ms/step - loss: 1.5667 - acc: 0.3025 - val_loss: 1.5421 - val_acc: 0.3273
Epoch 3/35
6/6 [==============================] - 4s 714ms/step - loss: 1.5191 - acc: 0.3433 - val_loss: 1.5387 - val_acc: 0.2927
Epoch 4/35
6/6 [==============================] - 4s 735ms/step - loss: 1.4997 - acc: 0.3460 - val_loss: 1.5335 - val_acc: 0.3309
Epoch 5/35
6/6 [==============================] - 3s 494ms/step - loss: 1.5293 - acc: 0.3515 - val_loss: 1.5361 - val_acc: 0.3400
Epoch 6/35
6/6 [==============================] - 4s 697ms/step - loss: 1.5210 - acc: 0.3515 - val_loss: 1.5091 - val_acc: 0.3145
Epoch 7/35
6/6 [==============================] - 3s 527ms/step - loss: 1.4800 - acc: 0.3978 - val_loss: 1.5227 - val_acc: 0.3036
Epoch 8/35
6/6 [==============================] - 3s 517ms/step - loss: 1.4849 - acc: 0.35

### Evaluation

In [21]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 1s 108ms/step - loss: 1.5091 - acc: 0.3145
